### **DLT Pipeline**

In [0]:
import dlt
from pyspark.sql.functions import *

**Streaming Table**

In [0]:
# Expectations
my_rules = {
    "rule1" : "product_id IS NOT NULL",
    "rule2" : "product_name IS NOT NULL"
}

In [0]:
@dlt.table(name="DimProducts_stage")
@dlt.expect_all_or_drop(my_rules)
def DimProducts_stage():
    df = spark.readStream.option("ignoreChanges", "true").table("databricks_cata.silver.products_silver")
    # Add a timestamp column for change tracking
    df = df.withColumn("processed_timestamp", current_timestamp())
    return df

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-5091606351914735>, line 1
----> 1 @dlt.table()
      2 
      3 def DimProducts_stage():
      4     df = spark.readStream.table("databricks_cata.silver_products_silver")
      5     return df

NameError: name 'dlt' is not defined

**Streaming View**

In [0]:
@dlt.view(name="DimProducts_view")
def DimProducts_view():
    df = spark.readStream.table("LIVE.DimProducts_stage")
    return df

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-5091606351914748>, line 1
----> 1 @dlt.view
      2 
      3 def DimProducts_view():
      4   df = spark.readStream.table("Live.DimProducts_stage")
      5   return df

NameError: name 'dlt' is not defined

**DimProducts**

In [0]:
dlt.create_streaming_table("DimProducts")

dlt.apply_changes(
    target="DimProducts",
    source="DimProducts_view",
    keys=["product_id"],
    sequence_by="processed_timestamp",  # Using the timestamp we added
    stored_as_scd_type=2
)